# Рекомендация тарифов

## Описание проекта

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — она уже сделана.

Задача: построить модель с максимально большим значением *accuracy*. Нужно довести долю правильных ответов по крайней мере до 0.75.

## Изучение данных

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [ ]:
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')

In [ ]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
df.duplicated().sum()

0

Итого: Пропусков и дубликатов в данных нет, т.к. была проведена предварительная предобработка

## Разделение данных на выборки

In [ ]:
seed = 4

In [ ]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [ ]:
l = df.shape[0]
indices = [int(.6*l), int(.8*l)] # 60% : 20% : 20%

features_train, features_valid, features_test = \
    np.split(features.sample(frac=1, random_state=seed), indices)
target_train, target_valid, target_test = \
    np.split(target.sample(frac=1, random_state=seed), indices)

In [ ]:
features_train.shape, target_train.shape

((1928, 4), (1928,))

In [ ]:
features_valid.shape, target_valid.shape

((643, 4), (643,))

In [ ]:
features_test.shape, target_test.shape

((643, 4), (643,))

## Исследование моделей

In [ ]:
best_accuracy = 0
best_depth = None
for depth in range(1, 21):
    model = DecisionTreeClassifier(random_state=seed, max_depth=depth)
    model.fit(features_train, target_train)
    accuracy = model.score(features_valid, target_valid)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_depth = depth

print(f'best depth: {best_depth}, accuracy: {best_accuracy:.3f}')

best depth: 6, accuracy: 0.807


In [ ]:
best_accuracy = 0
best_est = None
best_depth = None
for est in range(10, 101, 10):
    for depth in range(1, 11, 1):
        model = RandomForestClassifier(random_state=seed, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        accuracy = model.score(features_valid, target_valid)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_est = est
            best_depth = depth

print(f'best estimators: {best_est}, depth: {best_depth}, accuracy: {best_accuracy:.3f}')

best estimators: 50, depth: 7, accuracy: 0.813


In [ ]:
model = LogisticRegression(random_state=seed)
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)

print(f'accuracy: {accuracy:.3f}')

accuracy: 0.760


Самая высокую точность показала модель случайного леса, она же самая медленная. Чуть-чуть меньше точность у более быстрой модели решающего дерева.

Самая низкая точность у получилась у Логистической регрессии.

## Проверка модели на тестовой выборке

In [ ]:
model = RandomForestClassifier(random_state=seed, n_estimators=50, max_depth=7)
model.fit(features_train, target_train)
test_accuracy = model.score(features_test, target_test)

print(f'accuracy: {test_accuracy:.3f}')

accuracy: 0.801


## Проверка модели на адекватность

In [ ]:
dummy_accuracy = 0
for s in [0, 1, 'uniform', 'stratified']:
    if isinstance(s, int):
        model = DummyClassifier(strategy='constant', constant=s)
    else:
        model = DummyClassifier(random_state=seed, strategy=s)
    model.fit(features_train, target_train)
    accuracy = model.score(features_test, target_test)
    if accuracy > dummy_accuracy:
        dummy_accuracy = accuracy

print(f'Точность dummy модели: {dummy_accuracy:.3f}')
if test_accuracy > dummy_accuracy:
    print('Модель прошла проверку.')
else:
    print('Модель провалила проверку.')

Точность dummy модели: 0.689
Модель прошла проверку.


## Выводы

- Были построены несколько моделей классификации и подобраны гиперпараметры для них на валидации
- Выбрана лучшая модель и проверена точность на тестовой выборке
- Проведено сравнение с dummy моделью
- В результате, лучшая модель добилась необходимого качества